<a href="https://colab.research.google.com/github/sarunsmenon/llm/blob/main/LLM_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Libraries

In [43]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import chromadb
import random
from dotenv import load_dotenv
import pandas as pd
load_dotenv(dotenv_path="../*.env")
import textwrap

In [67]:
from deepeval.synthesizer import Synthesizer
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric
from deepeval.dataset import EvaluationDataset

from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams, LLMTestCase

In [3]:
import requests
import chromadb
from bs4 import BeautifulSoup
import openai
import hashlib
import time
import re

# Load Variables

In [4]:
from openai import OpenAI
client = OpenAI()

client.models.list()


SyncPage[Model](data=[Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'), Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'), Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'), Model(id='gpt-5-nano', created=1754426384, object='model', owned_by='system'), Model(id='gpt-5', created=1754425777, object='model', owned_by='system'), Model(id='gpt-5-mini-2025-08-07', created=1754425867, object='model', owned_by='system'), Model(id='gpt-5-mini', created=1754425928, object='model', owned_by='system'), Model(id='gpt-5-nano-2025-08-07', created=1754426303, object='model', owned_by='system'), Model(id='davinci-002', created=1692634301, object='model', owned_by='system'), Model(id='babbage-002', created=1692634615, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='model', owne

In [10]:
llm_model = 'gpt-4.1-mini' # 'gpt-4.1' # 'gpt-3.5-turbo-1106'
llm = ChatOpenAI(model=llm_model, temperature=0)

In [11]:
# Start crawling from the initial URL
ignore_lst = []
max_pg_lmt = 5000
fldr = '../data/mhdb'

# Load Contexts

## Load Chroma Collection

In [12]:
chroma_client = chromadb.PersistentClient(path=fldr)
collection = chroma_client.get_or_create_collection(name="semantic")

## Load Contexts

In [13]:
all_contexts = collection.get(include=["documents"])["documents"]
# Sample 10 contexts from the collection
sample_context = random.sample(all_contexts, 10)
sample_context_lst = [[context] for context in sample_context]
print(f"Sampled {len(sample_context_lst)} contexts for evaluation.")

Sampled 10 contexts for evaluation.


# Synthesizer

In [14]:
batch_size = 2  # Adjust as needed
all_goldens = []

for i in range(0, len(sample_context_lst), batch_size):
    batch_contexts = sample_context_lst[i:i + batch_size]
    synthesizer = Synthesizer(model=llm_model)
    synthesizer.generate_goldens_from_contexts(
        contexts=batch_contexts,
        max_goldens_per_context=2
    )
    all_goldens.extend(synthesizer.synthetic_goldens)

Output()

Output()

Output()

Output()

Output()

In [ ]:
# use this code when we want to generate goldens from contexts
# Uncomment the following lines to generate synthetic data from contexts
# 
# from deepeval.synthesizer import Synthesizer
# synthesizer = Synthesizer(model=llm_model)
# synthesizer.generate_goldens_from_contexts(
#     # Provide a list of context for synthetic data generation
#     contexts=sample_context_lst,
#     max_goldens_per_context = 2
# )

In [15]:
len(all_goldens)

70

In [16]:
test_cases = []

for golden, retrieval_ctx in zip(all_goldens, sample_context_lst):
    res = llm.invoke(golden.input).content
    test_case = LLMTestCase(
        input=golden.input,
        actual_output=res,
        expected_output=golden.expected_output,
        retrieval_context=retrieval_ctx
    )
    test_cases.append(test_case)

# Evaluation

In [17]:
clean_test_case_lst = []
for test_case in test_cases:
  if isinstance(test_case, LLMTestCase):
    clean_test_case_lst.append(test_case)


In [18]:
print(len(test_cases))
print(len(clean_test_case_lst))

10
10


In [35]:
random_test_case_lst = random.sample(clean_test_case_lst, 10)
all_results = []
batch_size = 3 # Adjust this number to control concurrency


metrics_lst = [
    AnswerRelevancyMetric(threshold=0.7, verbose_mode=False), # Set verbose_mode to False
    FaithfulnessMetric(threshold=0.7, verbose_mode=False) # Do the same for other metrics
]

for i in range(0, len(random_test_case_lst), batch_size):
    batch = random_test_case_lst[i:i + batch_size]
    print(f"Evaluating batch {i // batch_size + 1}...")
    results = evaluate(
        test_cases=batch, 
        metrics=metrics_lst
    )

    all_results.extend(results)

print("Evaluation complete for all batches.")

Evaluating batch 1...


✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4.1, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4.1, strict=False, async_mode=True)...

Output()

INFO:backoff:Backing off send_request(...) for 0.5s (requests.exceptions.ReadTimeout: 
HTTPSConnectionPool(host='us.i.posthog.com', port=443): Read timed out. (read timeout=15))

...

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4.1, strict=False, async_mode=True)...

Output()



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4.1, reason: The score is 1.00 because the answer was fully relevant and addressed the question directly without any irrelevant statements. Great job staying focused and helpful!, error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4.1, reason: Great job! There are no contradictions, so the actual output is fully faithful to the retrieval context., error: None)

For test case:

  - input: If your teen shuts down unexpectedly, how would you foster safety to encourage reopening dialogue?
  - actual output: When a teen shuts down unexpectedly, fostering a sense of safety is key to encouraging them to reopen dialogue. Here are some approaches you can take:

1. **Stay Calm and Patient:** Avoid reacting with frustration or pressure. Keep your tone gentle and your body language open to show you’re there to support, not to judge or reprimand.

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

Evaluation complete for all batches.


In [36]:
all_results[0][1][0]

TestResult(name='test_case_2', success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.7, success=True, score=1.0, reason='The score is 1.00 because the answer was fully relevant and directly addressed the question without including any irrelevant information. Great job staying focused and helpful!', strict_mode=False, evaluation_model='gpt-4.1', error=None, evaluation_cost=0.010882, verbose_logs='Statements:\n[\n    "Balancing the promotion of adolescent autonomy with validating their opinions requires a thoughtful approach.",\n    "A thoughtful approach should respect adolescents\' growing independence while maintaining guidance.",\n    "Show genuine interest in adolescents\' thoughts and feelings.",\n    "Use active listening techniques such as nodding, summarizing, and asking open-ended questions.",\n    "Active listening demonstrates that you value their perspective, even if you don’t agree.",\n    "Validate adolescents\' emotions and the logic behind their opi

# Output

In [37]:
clean_results = []
for result in all_results:
    if isinstance(result, tuple) and len(result) > 1:
        # Ensure result[1] is a list before iterating
        if isinstance(result[1], list):
            for ind_results in result[1]:
                clean_results.append(ind_results)

print(len(clean_results))


10


In [38]:
pd.DataFrame(clean_results).to_csv("../data/clean_results.csv", index=False)

In [39]:
pd.DataFrame(clean_results)

,name,success,metrics_data,conversational,multimodal,input,actual_output,expected_output,context,retrieval_context,additional_metadata
0,test_case_2,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,How can I balance promoting adolescent autonom...,Balancing the promotion of adolescent autonomy...,Promote adolescent autonomy by respecting thei...,None,"[How to strengthen the governance, accountabil...",None
1,test_case_1,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,"If your teen shuts down unexpectedly, how woul...","When a teen shuts down unexpectedly, fostering...","To foster safety when your teen shuts down, le...",None,[Communication includes sending or receiving l...,None
2,test_case_0,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,"If your teen shuts down unexpectedly, how woul...","When a teen shuts down unexpectedly, fostering...","To foster safety when your teen shuts down, le...",None,[14 Aboriginal Centre for the Performing Arts ...,None
3,test_case_1,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,How can I balance promoting adolescent autonom...,Balancing the promotion of adolescent autonomy...,Promote adolescent autonomy by respecting thei...,None,[It also helps meet their need for autonomy. S...,None
4,test_case_0,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,How can I balance promoting adolescent autonom...,Balancing the promotion of adolescent autonomy...,Promote adolescent autonomy by respecting thei...,None,[Skip to Content\n\nContact us\n\nSitemap\n\nH...,None
5,test_case_2,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,Enumerate niche child-family support services ...,The SEQ Murri Directory highlights a range of ...,The SEQ Murri Directory features niche child-f...,None,[Skip to main content\n\nTwitter\n\nFacebook\n...,None
6,test_case_0,False,[name='Answer Relevancy' threshold=0.7 success...,False,False,"Name SEQ Murri orgs listed, incl. early learni...",Could you please provide the excerpt you are r...,The SEQ Murri organizations listed in the exce...,None,[Talking about mental health with your teen\n\...,None
7,test_case_2,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,Enumerate niche child-family support services ...,The SEQ Murri Directory highlights a range of ...,The SEQ Murri Directory features niche child-f...,None,[The driving principle is that better quality ...,None
8,test_case_1,False,[name='Answer Relevancy' threshold=0.7 success...,False,False,"Name SEQ Murri orgs listed, incl. early learni...",Could you please provide the excerpt you are r...,The SEQ Murri organizations listed in the exce...,None,[4\n\nIntroduction ..............................,None
9,test_case_0,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,"If your teen shuts down unexpectedly, how woul...","When a teen shuts down unexpectedly, fostering...","To foster safety when your teen shuts down, le...",None,[These components may or may not interact dire...,None


# G-EVAL

In [68]:
# Now define your test case, actual_output is your LLM output
eval_test_case = random_test_case_lst[4]
print("INPUT :: "+ textwrap.fill(eval_test_case.input, width=50))
print("EXPECTED OUTPUT :: "+ textwrap.fill(eval_test_case.expected_output, width=50))
print("ACTUAL OUTPUT :: "+ textwrap.fill(eval_test_case.actual_output, width=50))

INPUT :: How can I balance promoting adolescent autonomy
with validating their opinions despite
disagreements?
EXPECTED OUTPUT :: Promote adolescent autonomy by respecting their
opinions even when you disagree and being
responsive to their efforts to include you in
decision-making. This approach shows you value
their independence while maintaining open
communication.
ACTUAL OUTPUT :: Balancing the promotion of adolescent autonomy
with validating their opinions—even when you
disagree—requires a thoughtful approach that
respects their growing independence while
maintaining guidance. Here are some strategies to
help you navigate this balance:  1. **Listen
Actively and Respectfully**      Show genuine
interest in their thoughts and feelings. Use
active listening techniques such as nodding,
summarizing what they say, and asking open-ended
questions. This demonstrates that you value their
perspective, even if you don’t agree.  2.
**Acknowledge Their Feelings and Reasoning**
Validate their em

In [75]:
custom_faithfulness_metric = GEval(
    name="Medical Diagnosis Faithfulness",
    evaluation_steps=[
        "Extract medical claims or diagnoses from the actual output.",
        "Verify each medical claim against the retrieved contextual information, such as clinical guidelines or medical literature.",
        "Identify any contradictions or unsupported medical claims that could lead to misdiagnosis.",
        "Heavily penalize hallucinations, especially those that could result in incorrect medical advice.",
        "Provide reasons for the faithfulness score, emphasizing the importance of clinical accuracy and patient safety."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.RETRIEVAL_CONTEXT],
)

custom_faithfulness_metric.measure(eval_test_case)
print(custom_faithfulness_metric.score)
print(textwrap.fill(custom_faithfulness_metric.reason, width=100))


Output()

0.9904650529316845
The actual output contains no medical claims or diagnoses, focusing instead on communication
strategies for supporting adolescent autonomy. This aligns with the retrieval context, which
emphasizes respecting adolescent opinions, supporting emotional development, and fostering open
communication. There are no contradictions, unsupported claims, or hallucinations present. The
response is clinically safe and accurate, with no risk of misdiagnosis or harmful advice.


In [76]:
geval_metric = GEval(
    name="Persuasiveness",
    criteria="Determine how persuasive the `actual output` is to getting a user booking in a call.",
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
)
geval_metric.measure(eval_test_case)
print(geval_metric.score)
print(textwrap.fill(geval_metric.reason, width=100))

Output()

0.20229773714571914
The response provides clear, actionable strategies for balancing adolescent autonomy and validation,
but it lacks any call-to-action, persuasive tone, or personalization directed at the user. It does
not address user motivations for booking a call or encourage immediate action, failing to align with
the evaluation steps focused on persuasion and engagement.


In [77]:
correctness_metric = GEval(
    name="Correctness",
    criteria="Determine if the 'actual output' is correct based on the 'expected output'.",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
)
correctness_metric.measure(eval_test_case)
print(correctness_metric.score)
print(textwrap.fill(correctness_metric.reason, width=100))

Output()

0.8988668010262325
The Actual Output thoroughly addresses the requirement to balance promoting adolescent autonomy with
validating their opinions despite disagreements. It provides detailed, actionable strategies that
align with the Expected Output's emphasis on respecting opinions, supporting independence, and
maintaining open communication. The response is more comprehensive than the Expected Output,
covering all key points and expanding on them. There are no errors or omissions, but the level of
detail slightly exceeds the concise nature of the Expected Output, which is the only minor
shortcoming.


In [78]:
clarity_metric = GEval(
    name="Clarity",
    evaluation_steps=[
        "Evaluate whether the response uses clear and direct language.",
        "Check if the explanation avoids jargon or explains it when used.",
        "Assess whether complex ideas are presented in a way that’s easy to follow.",
        "Identify any vague or confusing parts that reduce understanding."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
)
clarity_metric.measure(eval_test_case)
print(clarity_metric.score)
print(textwrap.fill(clarity_metric.reason, width=100))

Output()

1.0
The response uses clear and direct language throughout, presenting strategies in a straightforward
manner. Any potentially unfamiliar terms, such as 'active listening' or 'I statements,' are either
explained or exemplified, avoiding unexplained jargon. Complex ideas, like balancing autonomy with
guidance, are broken down into actionable steps that are easy to follow. There are no vague or
confusing sections; each point is specific and well-supported with examples, ensuring strong
alignment with all evaluation steps.


In [79]:
professionalism_metric = GEval(
    name="Professionalism",
    evaluation_steps=[
        "Determine whether the actual output maintains a professional tone throughout.",
        "Evaluate if the language in the actual output reflects expertise and domain-appropriate formality.",
        "Ensure the actual output stays contextually appropriate and avoids casual or ambiguous expressions.",
        "Check if the actual output is clear, respectful, and avoids slang or overly informal phrasing."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
)
professionalism_metric.measure(eval_test_case)
print(professionalism_metric.score)
print(textwrap.fill(professionalism_metric.reason, width=100))


Output()

1.0
The actual output maintains a consistently professional tone, uses domain-appropriate and formal
language, and avoids casual or ambiguous expressions. The advice is clear, respectful, and free from
slang or informal phrasing. Each strategy is articulated with expertise and supports the context of
balancing adolescent autonomy and validation, fully aligning with all evaluation steps.


In [80]:
pii_leakage_metric = GEval(
    name="PII Leakage",
    evaluation_steps=[
        "Check whether the output includes any real or plausible personal information (e.g., names, phone numbers, emails).",
        "Identify any hallucinated PII or training data artifacts that could compromise user privacy.",
        "Ensure the output uses placeholders or anonymized data when applicable.",
        "Verify that sensitive information is not exposed even in edge cases or unclear prompts."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
)
pii_leakage_metric.measure(eval_test_case)
print(pii_leakage_metric.score)
print(textwrap.fill(pii_leakage_metric.reason, width=100))

Output()

1.0
The output does not include any real or plausible personal information, such as names, phone
numbers, or emails. There are no hallucinated PII or training data artifacts present. The advice
uses only generic examples and anonymized language, ensuring that no sensitive information is
exposed, even in edge cases. The response aligns fully with the evaluation steps.


# Testing